# Using Gemini's Function Calling

## *Parallel calling*

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
June 2025

-----

## Introduction

This notebook shows how to do [Function Calling](https://ai.google.dev/gemini-api/docs/function-calling) workflows with Large Language Models (LLMs) of Gemini. 

(The Raku package ["WWW::Gemini"](https://github.com/antononcube/Raku-WWW-Gemini) is used.)

-----

## Setup

Load packages:

In [1]:
use WWW::Gemini;
use JSON::Fast;
use LLM::Tooling;

Choose a model:

In [2]:
my $model = "gemini-2.0-flash";
#my $model = "gemini-2.5-flash-preview-05-20";

gemini-2.0-flash

------

## Workflow

### Define a local function

Define a few _tools_:

In [4]:
#| Powers the spinning disco ball.
sub power-disco-ball-impl(
    Int:D $power #= Whether to turn the disco ball on or off.
    ) returns Hash {
    return { status => "Disco ball powered " ~ ($power ?? 'on' !! 'off') };
}
#= A status dictionary indicating the current state.

#| Play some music matching the specified parameters.
sub start-music-impl(
    Int:D $energetic, #=  Whether the music is energetic or not.
    Int:D $loud       #= Whether the music is loud or not.
    ) returns Hash {
    my $music-type = $energetic ?? 'energetic' !! 'chill';
    my $volume = $loud ?? 'loud' !! 'quiet';
    return { music_type => $music-type, volume => $volume };
    #= A dictionary containing the music settings.
}

#| Dim the lights.
sub dim-lights-impl(
    Numeric:D $brightness #= The brightness of the lights, 0.0 is off, 1.0 is full.
    ) returns Hash {
    return { brightness => $brightness };
}
#= A dictionary containing the new brightness setting.

&dim-lights-impl

In [5]:
#llm-tool-definition(&power-disco-ball-impl, format => 'raku')<function>.raku
#llm-tool-definition(&start-music-impl, format => 'raku')<function>.raku
llm-tool-definition(&dim-lights-impl, format => 'raku')<function>.raku

${:description("Dim the lights."), :name("dim-lights-impl"), :parameters(${:additionalProperties(Bool::False), :properties(${"\$brightness" => ${:description("The brightness of the lights, 0.0 is off, 1.0 is full."), :type("number")}}), :required($["\$brightness"]), :type("object")}), :strict(Bool::True), :type("function")}

These are the tool descriptions to be communicated to Gemini.

In [6]:
my @tools =
{
    :name("power-disco-ball-impl"), 
    :description("Powers the spinning disco ball."), 
    :parameters(
        {
            :type("object")
            :properties( {"\$power" => {:description("Whether to turn the disco ball on or off."), :type("integer")}}), 
            :required(["\$power"]), 
        }), 
},
{
    :name("start-music-impl"), 
    :description("Play some music matching the specified parameters."), 
    :parameters(
        {
            :type("object")
            :properties({
                "\$energetic" => {:description("Whether the music is energetic or not."), :type("integer")}, 
                "\$loud" => {:description("Whether the music is loud or not."), :type("integer")}
            }), 
            :required(["\$energetic", "\$loud"]), 
        }),
},
{
    :name("dim-lights-impl"), 
    :description("Dim the lights."), 
    :parameters(
        {
            :type("object")
            :properties({"\$brightness" => {:description("The brightness of the lights, 0.0 is off, 1.0 is full."), :type("number")}}), 
            :required(["\$brightness"]), 
        }), 
};

[{description => Powers the spinning disco ball., name => power-disco-ball-impl, parameters => {properties => {$power => {description => Whether to turn the disco ball on or off., type => integer}}, required => [$power], type => object}} {description => Play some music matching the specified parameters., name => start-music-impl, parameters => {properties => {$energetic => {description => Whether the music is energetic or not., type => integer}, $loud => {description => Whether the music is loud or not., type => integer}}, required => [$energetic $loud], type => object}} {description => Dim the lights., name => dim-lights-impl, parameters => {properties => {$brightness => {description => The brightness of the lights, 0.0 is off, 1.0 is full., type => number}}, required => [$brightness], type => object}}]

Here are additional tool-mode configurations:

In [7]:
my %toolConfig =
  functionCallingConfig => {
    mode => "ANY",
    allowedFunctionNames => <power-disco-ball-impl start-music-impl dim-lights-impl>
  };

{functionCallingConfig => {allowedFunctionNames => (power-disco-ball-impl start-music-impl dim-lights-impl), mode => ANY}}

### First communication with Gemini

In [8]:
# User prompt
my $prompt = 'Turn this place into a party!';

# Prepare the API request payload
my @messages = [{role => 'user',parts => [ %( text => $prompt ) ]}, ];

[{parts => [text => Turn this place into a party!], role => user}]

Make the first Gemini API call:

In [9]:
my $response = gemini-generate-content(
    @messages,
    :$model,
    :@tools,
    :%toolConfig
);

{candidates => [{avgLogprobs => -0.0012976408004760742, content => {parts => [{functionCall => {args => {$energetic => 1, $loud => 1}, name => start-music-impl}} {functionCall => {args => {$power => 1}, name => power-disco-ball-impl}} {functionCall => {args => {$brightness => 0.5}, name => dim-lights-impl}}], role => model}, finishReason => STOP, safetyRatings => [{category => HARM_CATEGORY_HATE_SPEECH, probability => NEGLIGIBLE} {category => HARM_CATEGORY_DANGEROUS_CONTENT, probability => NEGLIGIBLE} {category => HARM_CATEGORY_HARASSMENT, probability => NEGLIGIBLE} {category => HARM_CATEGORY_SEXUALLY_EXPLICIT, probability => NEGLIGIBLE}]}], modelVersion => gemini-2.0-flash, responseId => CgFFaOybFtWTmecPuOXGmAg, usageMetadata => {candidatesTokenCount => 30, candidatesTokensDetails => [{modality => TEXT, tokenCount => 30}], promptTokenCount => 113, promptTokensDetails => [{modality => TEXT, tokenCount => 113}], totalTokenCount => 143}}

The response is already parsed from JSON to Raku. Here is its JSON form:

In [10]:
to-json($response)

{
  "modelVersion": "gemini-2.0-flash",
  "candidates": [
    {
      "content": {
        "role": "model",
        "parts": [
          {
            "functionCall": {
              "name": "start-music-impl",
              "args": {
                "$loud": 1,
                "$energetic": 1
              }
            }
          },
          {
            "functionCall": {
              "args": {
                "$power": 1
              },
              "name": "power-disco-ball-impl"
            }
          },
          {
            "functionCall": {
              "args": {
                "$brightness": 0.5
              },
              "name": "dim-lights-impl"
            }
          }
        ]
      },
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE"
        },
        {
          "probability": "NEGLIGIBLE",
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT"
        },
        {
          "cat

### Refine the response with functional calls

The following copy of the messages is not required, but it makes repeated experiments easier:

In [11]:
my @messages2 = @messages;

[{parts => [text => Turn this place into a party!], role => user}]

Let us define `LLM::Tool` object for each tool:

In [12]:
my @toolObjects = [&power-disco-ball-impl, &start-music-impl, &dim-lights-impl].map({ LLM::Tool.new($_) })

[LLMTool(power-disco-ball-impl, Powers the spinning disco ball.) LLMTool(start-music-impl, Play some music matching the specified parameters.) LLMTool(dim-lights-impl, Dim the lights.)]

Make a `LLM::Request` object for each request from the (first) LLM response:

In [13]:
my @requestObjects = $response<candidates>»<content>»<parts>.&flatten»<functionCall>.map({ LLM::ToolRequest.new( $_<name>, $_<args>) })

[LLMToolRequest(start-music-impl, :$energetic(1), :$loud(1), :id(Whatever)) LLMToolRequest(power-disco-ball-impl, :$power(1), :id(Whatever)) LLMToolRequest(dim-lights-impl, :$brightness(0.5), :id(Whatever))]

In [14]:
.say for @requestObjects.map({ generate-llm-tool-response(@toolObjects, $_) })».output

{music_type => energetic, volume => loud}
{status => Disco ball powered on}
{brightness => 0.5}


In [15]:
.say for @requestObjects.map({ generate-llm-tool-response(@toolObjects, $_) })».Hash('Gemini')

{functionResponse => {name => start-music-impl, response => {content => {music_type => energetic, volume => loud}}}}
{functionResponse => {name => power-disco-ball-impl, response => {content => {status => Disco ball powered on}}}}
{functionResponse => {name => dim-lights-impl, response => {content => {brightness => 0.5}}}}


Process the response:
- Make a request object for each function call request
- Compute the tool results
- Form corresponding user message with those results
- Send the messages to the LLM

In [ ]:
my $assistant-message = $response<candidates>[0]<content>;
if $assistant-message<parts> {

    # Find function call parts and make corresponding tool objects
    my @requestObjects;
    for |$assistant-message<parts> -> %part {
        if %part<functionCall> {
            @requestObjects.push: LLM::ToolRequest.new( %part<functionCall><name>, %part<functionCall><args> ) 
        }
    }    

    # Add assistance message
    @messages2.push($assistant-message);

    # Compute tool responses
    my @funcParts = @requestObjects.map({ generate-llm-tool-response(@toolObjects, $_) })».Hash('Gemini');

    # Make and add the user response
    my %function-response =
        role => 'user',
        parts => @funcParts;

    @messages2.push(%function-response);
                
    # Send the second request with function result
    my $final-response = gemini-generate-content(
        @messages2,
        :@tools,
        :$model,
        format => "raku"
    );
                
    say "Assistant: ", $final-response<candidates>[0]<content><parts>».<text>.join("\n");

} else {
    say "Assistant: $assistant-message<content>";
}

Assistant: Alright! I've started some energetic and loud music, turned on the disco ball, and dimmed the lights to 50% brightness. Let's get this party started!

